# Download the Sample Data
Using the Python code below, we can download a land cover raster file for Johnson County, Iowa and a shapefile for the city boundary of Iowa City. We will use these two files to test the zonal statistics tool that we will create in the following steps. For this test, we will use our zonal statistics tool, along with the files below and a user-supplied raster class range to calculate the percentage of Iowa City covered by tree canopy.

In [ ]:
import zipfile, urllib.request, shutil

url1 = 'https://iowageodata.s3.amazonaws.com/imageryBaseMapsEarthCover/earthCover/Land_cover_2009_1m/HRLC_2009_52.zip'
url2 = 'https://github.com/ui-libraries/Zonal_Statistics_Tool_JupyterNotebook/raw/main/IowaCity_Shapefile.zip'
file_name1 = 'HRLC_2009_52.zip'
file_name2 = 'IowaCity_Shapefile.zip'

with urllib.request.urlopen(url1) as response, open(file_name1, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
    with zipfile.ZipFile(file_name1) as zf:
        zf.extractall()

with urllib.request.urlopen(url2) as response, open(file_name2, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
    with zipfile.ZipFile(file_name2) as zf:
        zf.extractall()

# Install Required Packages
If you do not already have gdal, rasterio, rasterstats, ipywidgets, and ipyfilechooser installed, you can install them by running the following code. Just as a warning, this cell will take a few minutes to run. If you need to grab some coffee or a snack, these installations should be finished by the time you return.

In [ ]:
import sys
!conda install -c conda-forge --yes --prefix {sys.prefix} gdal
!conda install -c conda-forge --yes --prefix {sys.prefix} rasterio
!conda install -c conda-forge --yes --prefix {sys.prefix} rasterstats
!conda install -c conda-forge --yes --prefix {sys.prefix} ipywidgets
!conda install -c conda-forge --yes --prefix {sys.prefix} ipyfilechooser

# Import Modules
Before running any scripts, we need to import the following Python modules, which will allow us to calculate zonal statistics for our chosen raster land cover file and selected land cover class range. If you are attempting to replicate this in your own Anaconda environment, again make sure to install gdal, rasterio, rasterstats, ipywidgets, and ipyfilechooser using the ```conda install -c conda-forge``` command in your Mac terminal or Windows shell before you attempt to import and use these modules. Or, run the code in the cell above to install these for this Jupyter Notebook. The gdal, rasterio, and rasterstats modules are necessary for our zonal stats analysis. The ipywidgets and ipyfilechooser modules are required for providing input option widgets in Jupyter Notebook.

In [ ]:
import os
from osgeo import gdal
from osgeo import ogr, osr
import numpy as np
from numpy import zeros
from numpy import logical_and
import rasterio as rio
from rasterstats import zonal_stats
from ipywidgets import widgets
from ipyfilechooser import FileChooser

print("Modules imported!\n-----")
print("Initializing code...")

# Build the Ipywidgets Input Cells for Jupyter Notebook
In the next section of Python code, we will add to our code above by building out the scaffolding that returns options to handle user inputs. The first four inputs will ask for the shapefile of the user's analysis area, the raster file containing the land cover data, the lowest value of the raster classes being analyzed, and the highest value of the raster classes being analyzed. Then, an "OK" button will initiate the analytical portion of the script, using the provided inputs. While we could use the Tkinter module to create the input GUI if we were running Python on our own desktop, we will be using Ipywidgets to handle inputs entirely within this shared Jupyter Notebook file. Run the following code and notice how the input form appears inside the cell. Don't worry about filling anything out at this time, just click "OK" and note that it returns the message, "Processing happens here!"

In [ ]:
import os
from osgeo import gdal
from osgeo import ogr, osr
import numpy as np
from numpy import zeros
from numpy import logical_and
import rasterio as rio
from rasterstats import zonal_stats
from ipywidgets import widgets
from ipyfilechooser import FileChooser

print("Modules imported!\n-----")
print("Initializing code...")

## Set up the user input form

## Return Shapefile Path
lbl1=widgets.Label('Input the path to the shapefile for your area of interest')
display(lbl1)
fc1=FileChooser()
display(fc1)

## Return Raster File Path
lbl2=widgets.Label('Input the path to the raster for zonal analysis')
display(lbl2)
fc2=FileChooser()
display(fc2)

## Return Lowest Value of Desired Raster Classes
lbl3=widgets.Label('Input the lowest value in the raster classification range you want to analyze')
display(lbl3)
text3=widgets.Text()
display(text3)

## Return Highest Value of Desired Raster Classes
lbl4=widgets.Label('Input the highest value in the raster classification range you want to analyze')
display(lbl4)
text4=widgets.Text()
display(text4)

## Run the Script with OK Button
button=widgets.Button(description='OK')
output=widgets.Output()
display(button, output)

def on_button_clicked(b):
    with output:
        print('Processing happens here!')

button.on_click(on_button_clicked)

# Define User Inputs
With the following code, we will build upon the previous code and define each of the user-provided inputs. This time, run the code and choose the City.shp file you downloaded for the input shapefile and the HRLC_2009_52.img file you downloaded for the input raster file. Then, input 3 for the lowest value in the raster classification range and 6 for the highest value. If you check the metadata that comes with the img file you unzipped, you will see that classes 3 to 6 contain all of the tree canopy classes. Click "OK" and the code should print the paths to your input data, as well as the raster class values that you provided.

In [ ]:
import os
from osgeo import gdal
from osgeo import ogr, osr
import numpy as np
from numpy import zeros
from numpy import logical_and
import rasterio as rio
from rasterstats import zonal_stats
from ipywidgets import widgets
from ipyfilechooser import FileChooser

print("Modules imported!\n-----")
print("Initializing code...")

## Set up the user input form

## Return Shapefile Path
lbl1=widgets.Label('Input the path to the shapefile for your area of interest')
display(lbl1)
fc1=FileChooser()
display(fc1)

## Return Raster File Path
lbl2=widgets.Label('Input the path to the raster for zonal analysis')
display(lbl2)
fc2=FileChooser()
display(fc2)

## Return Lowest Value of Desired Raster Classes
lbl3=widgets.Label('Input the lowest value in the raster classification range you want to analyze')
display(lbl3)
text3=widgets.IntText()
display(text3)

## Return Highest Value of Desired Raster Classes
lbl4=widgets.Label('Input the highest value in the raster classification range you want to analyze')
display(lbl4)
text4=widgets.IntText()
display(text4)

## Run the Script with OK Button
button=widgets.Button(description='OK')
output=widgets.Output()
display(button, output)

def on_button_clicked(b):
    with output:
        
        ##  Define input variables
        input_zone_polygon = fc1.selected
        input_value_raster = fc2.selected
        low_class_str = str(text3.value)
        high_class_str = str(text4.value)
        
        ## Convert numeric strings to integers
        low_class_int = int(low_class_str)
        high_class_int = int(high_class_str)
        
        ## Ensure all entries are filled
        if input_zone_polygon == "":
            print("Error: Select the shapefile for your area of interest")
        if input_value_raster == "":
            print("Error: Select the raster for zonal analysis")
        if low_class_int == "":
            print("Error: Input the lowest value in the raster classification range you want to analyze")
        if high_class_int == "":
            print("Error: Input the highest value in the raster classification range you want to analyze")
            
        ## Check user inputs
        print("The input shapefile is: " + input_zone_polygon + ". The input raster is: " + input_value_raster + ". The lowest raster class value is: " + low_class_str + ". The highest raster class value is: " + high_class_str + ".")

button.on_click(on_button_clicked)

# Check Raster File Extension
Now that we have confirmed that our input form works with the user-provided inputs, let's add some code to confirm whether our raster file conforms to the two file formats that this code accepts, img or tif. Run this script and complete the input form as before. This time, when you click "OK", you should see a message informing you that the input raster is in .img format.

In [ ]:
import os
from osgeo import gdal
from osgeo import ogr, osr
import numpy as np
from numpy import zeros
from numpy import logical_and
import rasterio as rio
from rasterstats import zonal_stats
from ipywidgets import widgets
from ipyfilechooser import FileChooser

print("Modules imported!\n-----")
print("Initializing code...")

## Set up the user input form

## Return Shapefile Path
lbl1=widgets.Label('Input the path to the shapefile for your area of interest')
display(lbl1)
fc1=FileChooser()
display(fc1)

## Return Raster File Path
lbl2=widgets.Label('Input the path to the raster for zonal analysis')
display(lbl2)
fc2=FileChooser()
display(fc2)

## Return Lowest Value of Desired Raster Classes
lbl3=widgets.Label('Input the lowest value in the raster classification range you want to analyze')
display(lbl3)
text3=widgets.IntText()
display(text3)

## Return Highest Value of Desired Raster Classes
lbl4=widgets.Label('Input the highest value in the raster classification range you want to analyze')
display(lbl4)
text4=widgets.IntText()
display(text4)

## Run the Script with OK Button
button=widgets.Button(description='OK')
output=widgets.Output()
display(button, output)

def on_button_clicked(b):
    with output:
        
        ##  Define input variables
        input_zone_polygon = fc1.selected
        input_value_raster = fc2.selected
        low_class_str = str(text3.value)
        high_class_str = str(text4.value)
        
        ## Convert numeric strings to integers
        low_class_int = int(low_class_str)
        high_class_int = int(high_class_str)
        
        ## Ensure all entries are filled
        if input_zone_polygon == "":
            print("Error: Select the shapefile for your area of interest")
        if input_value_raster == "":
            print("Error: Select the raster for zonal analysis")
        if low_class_int == "":
            print("Error: Input the lowest value in the raster classification range you want to analyze")
        if high_class_int == "":
            print("Error: Input the highest value in the raster classification range you want to analyze")
            
        print('Retrieving the directory path for the input raster.')

        ## Get the directory path of the input raster
        ras_dir_path = os.path.dirname(input_value_raster)

        print('Retrieving the file extension of the input raster.')

        ## Get the file extension of the raster file
        file_ext = os.path.splitext(input_value_raster)[1]

        print('Checking if the file extension of the input raster is tif or img.')

        if file_ext == '.tif':
            drive = 'GTiff'
        elif file_ext == '.img':
            drive = 'HFA'
        else:
            print('Error: The input raster file needs to be in .tif or .img format.')
                
        ## Print the raster file extension
        print('The input raster file extension is: ' + file_ext)
 
button.on_click(on_button_clicked)       

# Reclassify the Raster to a Binary Classification
If all went well in the previous step, we are ready to reclassify the input raster from its many raster classes to a simple binary classification, where 1 is equivalent to the tree canopy classes (3 - 6) that we chose, and 0 is equivalent to all other classes we wish to exclude from analysis. After running this cell, and about a minute of waiting, you should see the message, "Done reclassifying the raster" and a new output file called "raster2.img" alongside your other downloaded files. However, we need to reproject this raster to match the projection of the shapefile in order for our zonal stats analysis to be successful.

In [ ]:
import os
from osgeo import gdal
from osgeo import ogr, osr
import numpy as np
from numpy import zeros
from numpy import logical_and
import rasterio as rio
from rasterstats import zonal_stats
from ipywidgets import widgets
from ipyfilechooser import FileChooser

print("Modules imported!\n-----")
print("Initializing code...")

## Set up the user input form

## Return Shapefile Path
lbl1=widgets.Label('Input the path to the shapefile for your area of interest')
display(lbl1)
fc1=FileChooser()
display(fc1)

## Return Raster File Path
lbl2=widgets.Label('Input the path to the raster for zonal analysis')
display(lbl2)
fc2=FileChooser()
display(fc2)

## Return Lowest Value of Desired Raster Classes
lbl3=widgets.Label('Input the lowest value in the raster classification range you want to analyze')
display(lbl3)
text3=widgets.IntText()
display(text3)

## Return Highest Value of Desired Raster Classes
lbl4=widgets.Label('Input the highest value in the raster classification range you want to analyze')
display(lbl4)
text4=widgets.IntText()
display(text4)

## Run the Script with OK Button
button=widgets.Button(description='OK')
output=widgets.Output()
display(button, output)

def on_button_clicked(b):
    with output:
        
        ##  Define input variables
        input_zone_polygon = fc1.selected
        input_value_raster = fc2.selected
        low_class_str = str(text3.value)
        high_class_str = str(text4.value)
        
        ## Convert numeric strings to integers
        low_class_int = int(low_class_str)
        high_class_int = int(high_class_str)
        
        ## Ensure all entries are filled
        if input_zone_polygon == "":
            print("Error: Select the shapefile for your area of interest")
        if input_value_raster == "":
            print("Error: Select the raster for zonal analysis")
        if low_class_int == "":
            print("Error: Input the lowest value in the raster classification range you want to analyze")
        if high_class_int == "":
            print("Error: Input the highest value in the raster classification range you want to analyze")
            
        print('Retrieving the directory path for the input raster.')

        ## Get the directory path of the input raster
        ras_dir_path = os.path.dirname(input_value_raster)

        print('Retrieving the file extension of the input raster.')

        ## Get the file extension of the raster file
        file_ext = os.path.splitext(input_value_raster)[1]

        print('Checking if the file extension of the input raster is tif or img.')

        if file_ext == '.tif':
            drive = 'GTiff'
        elif file_ext == '.img':
            drive = 'HFA'
        else:
            print('Error: The input raster file needs to be in .tif or .img format.')
                
        print('Reclassifying the raster to a binary 0,1 classification...')
        
        #Define the gdal driver with the drive variable from the conditional test
        driver = gdal.GetDriverByName(drive)
        
        file = gdal.Open(input_value_raster)
        band = file.GetRasterBand(1)

        # reclassification
        classification_values = [0,low_class_int,high_class_int + 1]
        classification_output_values = [0,1,0]
        
        block_sizes = band.GetBlockSize()
        x_block_size = block_sizes[0]
        y_block_size = block_sizes[1]

        xsize = band.XSize
        ysize = band.YSize

        max_value = band.GetMaximum()
        min_value = band.GetMinimum()
        
        if max_value == None or min_value == None:
            stats = band.GetStatistics(0, 1)
            max_value = stats[1]
            min_value = stats[0]
        
        # create new file
        file2 = driver.Create( ras_dir_path + '/raster2' + file_ext, xsize , ysize , 1, gdal.GDT_Byte)
        
        # spatial ref system
        file2.SetGeoTransform(file.GetGeoTransform())
        file2.SetProjection(file.GetProjection())
        
        print('Reassigning raster values...please wait...')
        for i in range(0, ysize, y_block_size):
            if i + y_block_size < ysize:
                rows = y_block_size
            else:
                rows = ysize - i
            for j in range(0, xsize, x_block_size):
                if j + x_block_size < xsize:
                    cols = x_block_size
                else:
                    cols = xsize - j

                data = band.ReadAsArray(j, i, cols, rows)
                r = zeros((rows, cols), np.uint8)

                for k in range(len(classification_values) - 1):
                    if classification_values[k] <= max_value and (classification_values[k + 1] > min_value ):
                        r = r + classification_output_values[k] * logical_and(data >= classification_values[k], data < classification_values[k + 1])
                if classification_values[k + 1] < max_value:
                    r = r + classification_output_values[k+1] * (data >= classification_values[k + 1])

                file2.GetRasterBand(1).WriteArray(r,j,i)
        
        file2 = None
        
        print('Done reclassifying the raster.')
 
button.on_click(on_button_clicked)   

# Reproject the Raster to Match the Shapefile
Now, let's rerun the process with some additional lines of code that will create a new raster file in the same projection as our input shapefile. When this code finishes running, you should see a new raster file called "raster2_reproject.img" alongside the "raster2.img" file you created in the previous step. This step should take about 2.5 minutes to complete processing.

In [ ]:
import os
from osgeo import gdal
from osgeo import ogr, osr
import numpy as np
from numpy import zeros
from numpy import logical_and
import rasterio as rio
from rasterstats import zonal_stats
from ipywidgets import widgets
from ipyfilechooser import FileChooser

print("Modules imported!\n-----")
print("Initializing code...")

## Set up the user input form

## Return Shapefile Path
lbl1=widgets.Label('Input the path to the shapefile for your area of interest')
display(lbl1)
fc1=FileChooser()
display(fc1)

## Return Raster File Path
lbl2=widgets.Label('Input the path to the raster for zonal analysis')
display(lbl2)
fc2=FileChooser()
display(fc2)

## Return Lowest Value of Desired Raster Classes
lbl3=widgets.Label('Input the lowest value in the raster classification range you want to analyze')
display(lbl3)
text3=widgets.IntText()
display(text3)

## Return Highest Value of Desired Raster Classes
lbl4=widgets.Label('Input the highest value in the raster classification range you want to analyze')
display(lbl4)
text4=widgets.IntText()
display(text4)

## Run the Script with OK Button
button=widgets.Button(description='OK')
output=widgets.Output()
display(button, output)

def on_button_clicked(b):
    with output:
        
        ##  Define input variables
        input_zone_polygon = fc1.selected
        input_value_raster = fc2.selected
        low_class_str = str(text3.value)
        high_class_str = str(text4.value)
        
        ## Convert numeric strings to integers
        low_class_int = int(low_class_str)
        high_class_int = int(high_class_str)
        
        ## Ensure all entries are filled
        if input_zone_polygon == "":
            print("Error: Select the shapefile for your area of interest")
        if input_value_raster == "":
            print("Error: Select the raster for zonal analysis")
        if low_class_int == "":
            print("Error: Input the lowest value in the raster classification range you want to analyze")
        if high_class_int == "":
            print("Error: Input the highest value in the raster classification range you want to analyze")
            
        print('Retrieving the directory path for the input raster.')

        ## Get the directory path of the input raster
        ras_dir_path = os.path.dirname(input_value_raster)

        print('Retrieving the file extension of the input raster.')

        ## Get the file extension of the raster file
        file_ext = os.path.splitext(input_value_raster)[1]

        print('Checking if the file extension of the input raster is tif or img.')

        if file_ext == '.tif':
            drive = 'GTiff'
        elif file_ext == '.img':
            drive = 'HFA'
        else:
            print('Error: The input raster file needs to be in .tif or .img format.')
                
        print('Reclassifying the raster to a binary 0,1 classification...')
        
        #Define the gdal driver with the drive variable from the conditional test
        driver = gdal.GetDriverByName(drive)
        
        file = gdal.Open(input_value_raster)
        band = file.GetRasterBand(1)

        # reclassification
        classification_values = [0,low_class_int,high_class_int + 1]
        classification_output_values = [0,1,0]
        
        block_sizes = band.GetBlockSize()
        x_block_size = block_sizes[0]
        y_block_size = block_sizes[1]

        xsize = band.XSize
        ysize = band.YSize

        max_value = band.GetMaximum()
        min_value = band.GetMinimum()
        
        if max_value == None or min_value == None:
            stats = band.GetStatistics(0, 1)
            max_value = stats[1]
            min_value = stats[0]
        
        # create new file
        file2 = driver.Create( ras_dir_path + '/raster2' + file_ext, xsize , ysize , 1, gdal.GDT_Byte)
        
        # spatial ref system
        file2.SetGeoTransform(file.GetGeoTransform())
        file2.SetProjection(file.GetProjection())
        
        print('Reassigning raster values...please wait...')
        for i in range(0, ysize, y_block_size):
            if i + y_block_size < ysize:
                rows = y_block_size
            else:
                rows = ysize - i
            for j in range(0, xsize, x_block_size):
                if j + x_block_size < xsize:
                    cols = x_block_size
                else:
                    cols = xsize - j

                data = band.ReadAsArray(j, i, cols, rows)
                r = zeros((rows, cols), np.uint8)

                for k in range(len(classification_values) - 1):
                    if classification_values[k] <= max_value and (classification_values[k + 1] > min_value ):
                        r = r + classification_output_values[k] * logical_and(data >= classification_values[k], data < classification_values[k + 1])
                if classification_values[k + 1] < max_value:
                    r = r + classification_output_values[k+1] * (data >= classification_values[k + 1])

                file2.GetRasterBand(1).WriteArray(r,j,i)
        
        file2 = None
        
        print('Done reclassifying the raster. Reprojecting the raster to the shapefile projection...')
        print('Identifying the EPSG code of the input SHP')
            
        # Get the EPSG code of the input shapefile
        shp_driver = ogr.GetDriverByName('ESRI Shapefile')
        dataset = shp_driver.Open(input_zone_polygon)
        layer = dataset.GetLayer()
        spatialRef = layer.GetSpatialRef()
        shp_epsg = spatialRef.GetAttrValue("GEOGCS|AUTHORITY", 1)

        print('Shapefile projection is EPSG:' + shp_epsg + '.')
        print('Reprojecting the raster to match SHP...please wait...')
            
        # Reproject the raster
        input_raster = gdal.Open(ras_dir_path + '/raster2' + file_ext)
        output_raster = ras_dir_path + '/raster2_reproject' + file_ext

        warp = gdal.Warp(output_raster,input_raster,dstSRS='EPSG:'+str(shp_epsg))
        warp = None

        print('Done reprojecting')
 
button.on_click(on_button_clicked)   

# Compute Zonal Statistics
Now that we have matched the raster projection to the input shapefile projection, we can calculate zonal statistics for our selected raster classes within the area covered by the shapefile. In this case, we are calculating the percent of Iowa City covered by tree canopy. This is done by adding a few lines of code supported by the rasterstats module that we imported. What follows is the complete Python code for our zonal statistics tool. The entire process should take about 3 minutes. When it finishes, you should see a message informing you what percentage of the input shapefile (Iowa City) is covered by the selected raster classes (3, 4, 5, and 6).

In [ ]:
import os
from osgeo import gdal
from osgeo import ogr, osr
import numpy as np
from numpy import zeros
from numpy import logical_and
import rasterio as rio
from rasterstats import zonal_stats
from ipywidgets import widgets
from ipyfilechooser import FileChooser

print("Modules imported!\n-----")
print("Initializing code...")

## Set up the user input form

## Return Shapefile Path
lbl1=widgets.Label('Input the path to the shapefile for your area of interest')
display(lbl1)
fc1=FileChooser()
display(fc1)

## Return Raster File Path
lbl2=widgets.Label('Input the path to the raster for zonal analysis')
display(lbl2)
fc2=FileChooser()
display(fc2)

## Return Lowest Value of Desired Raster Classes
lbl3=widgets.Label('Input the lowest value in the raster classification range you want to analyze')
display(lbl3)
text3=widgets.IntText()
display(text3)

## Return Highest Value of Desired Raster Classes
lbl4=widgets.Label('Input the highest value in the raster classification range you want to analyze')
display(lbl4)
text4=widgets.IntText()
display(text4)

## Run the Script with OK Button
button=widgets.Button(description='OK')
output=widgets.Output()
display(button, output)

def on_button_clicked(b):
    with output:
        
        ##  Define input variables
        input_zone_polygon = fc1.selected
        input_value_raster = fc2.selected
        low_class_str = str(text3.value)
        high_class_str = str(text4.value)
        
        ## Convert numeric strings to integers
        low_class_int = int(low_class_str)
        high_class_int = int(high_class_str)
        
        ## Ensure all entries are filled
        if input_zone_polygon == "":
            print("Error: Select the shapefile for your area of interest")
        if input_value_raster == "":
            print("Error: Select the raster for zonal analysis")
        if low_class_int == "":
            print("Error: Input the lowest value in the raster classification range you want to analyze")
        if high_class_int == "":
            print("Error: Input the highest value in the raster classification range you want to analyze")
            
        print('Retrieving the directory path for the input raster.')

        ## Get the directory path of the input raster
        ras_dir_path = os.path.dirname(input_value_raster)

        print('Retrieving the file extension of the input raster.')

        ## Get the file extension of the raster file
        file_ext = os.path.splitext(input_value_raster)[1]

        print('Checking if the file extension of the input raster is tif or img.')

        if file_ext == '.tif':
            drive = 'GTiff'
        elif file_ext == '.img':
            drive = 'HFA'
        else:
            print('Error: The input raster file needs to be in .tif or .img format.')
                
        print('Reclassifying the raster to a binary 0,1 classification...')
            
        #Define the gdal driver with the drive variable from the conditional test
        driver = gdal.GetDriverByName(drive)
        
        file = gdal.Open(input_value_raster)
        band = file.GetRasterBand(1)

        # reclassification
        classification_values = [0,low_class_int,high_class_int + 1]
        classification_output_values = [0,1,0]
        
        block_sizes = band.GetBlockSize()
        x_block_size = block_sizes[0]
        y_block_size = block_sizes[1]

        xsize = band.XSize
        ysize = band.YSize

        max_value = band.GetMaximum()
        min_value = band.GetMinimum()
        
        if max_value == None or min_value == None:
            stats = band.GetStatistics(0, 1)
            max_value = stats[1]
            min_value = stats[0]
        
        # create new file
        file2 = driver.Create( ras_dir_path + '/raster2' + file_ext, xsize , ysize , 1, gdal.GDT_Byte)
        
        # spatial ref system
        file2.SetGeoTransform(file.GetGeoTransform())
        file2.SetProjection(file.GetProjection())
        
        print('Reassigning raster values...please wait...')
        for i in range(0, ysize, y_block_size):
            if i + y_block_size < ysize:
                rows = y_block_size
            else:
                rows = ysize - i
            for j in range(0, xsize, x_block_size):
                if j + x_block_size < xsize:
                    cols = x_block_size
                else:
                    cols = xsize - j

                data = band.ReadAsArray(j, i, cols, rows)
                r = zeros((rows, cols), np.uint8)

                for k in range(len(classification_values) - 1):
                    if classification_values[k] <= max_value and (classification_values[k + 1] > min_value ):
                        r = r + classification_output_values[k] * logical_and(data >= classification_values[k], data < classification_values[k + 1])
                if classification_values[k + 1] < max_value:
                    r = r + classification_output_values[k+1] * (data >= classification_values[k + 1])

                file2.GetRasterBand(1).WriteArray(r,j,i)
        
        file2 = None

        print('Done reclassifying the raster. Reprojecting the raster to the shapefile projection...')
        print('Identifying the EPSG code of the input SHP')
            
        # Get the EPSG code of the input shapefile
        shp_driver = ogr.GetDriverByName('ESRI Shapefile')
        dataset = shp_driver.Open(input_zone_polygon)
        layer = dataset.GetLayer()
        spatialRef = layer.GetSpatialRef()
        shp_epsg = spatialRef.GetAttrValue("GEOGCS|AUTHORITY", 1)

        print('Shapefile projection is EPSG:' + shp_epsg + '.')
        print('Reprojecting the raster to match SHP...please wait...')
            
        # Reproject the raster
        input_raster = gdal.Open(ras_dir_path + '/raster2' + file_ext)
        output_raster = ras_dir_path + '/raster2_reproject' + file_ext

        warp = gdal.Warp(output_raster,input_raster,dstSRS='EPSG:'+str(shp_epsg))
        warp = None

        print('Done reprojecting. Processing zonal statistics...')
            
        zs = zonal_stats(input_zone_polygon,output_raster,stats=['min', 'max', 'mean', 'count', 'sum'])

        ## Define each zonal stat
        min = [x['min'] for x in zs]
        max = [x['max'] for x in zs]
        mean = [x['mean'] for x in zs]
        count = [x['count'] for x in zs]
        sum = [x['sum'] for x in zs]

        ## Build the zonal stats message content
        lines = ["AOI covered by selected raster classes: " + str(round(mean[0]*100,2))+"%", "minimum: " + str(min[0]), "maximum: " + str(max[0]), "count: " + str(count[0]), "sum: " + str(sum[0])]

        ## Display the message content in separate lines
        print("\n".join(lines))

        print('All done processing!')
 
button.on_click(on_button_clicked)     

# Conclusion
When our zonal statistics tool finishes running, you should see that the selected canopy raster classes (3, 4, 5, and 6) constitute 29.26% of the area of interest (Iowa City). You can also tell that the analyzed raster represents a binary classification scheme, in that the minimum value is 0 and the maximum value is 1. Count refers to the total number of raster pixels in the shapefile-defined AOI, while sum refers to the total number of pixels within the selected raster classes. If you divide sum by count, you will get the percent coverage. This concludes the tutorial for creating a Python tool for computing zonal statistics from a raster land cover file and an area of interest shapefile. Feel free to copy and modify this code to serve your own interests!